<a href="https://colab.research.google.com/github/jjschueder/NLP/blob/main/JJS_Homework5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 5
## Joe Schueder

1.	Compile a list of static links (permalinks) to individual user movie reviews from one particular website. This will be your working dataset for this assignment, as well as for assignments 7 and 8, which together will make up your semester project.   
a.	It does not matter if you use a crawler or if you manually collect the links, but you will need at least 100 movie review links. Note that, as of this writing, the robots.txt file of IMDB.com allows the crawling of user reviews.  
b.	Each link should be to a web page that has only one user review of only one movie, e.g., the user review permalinks on the IMDB site.  
c.	Choose reviews of movies that are all in the same genre, e.g., sci-fi, mystery, romance, superhero, etc.    
d.	Make sure your collection includes reviews of several movies in your chosen genre and that it includes a mix of negative and positive reviews.    

2.	Extract noun phrase (NP) chunks from your reviews using the following procedure:  
a.	In Python, use BeautifulSoup to grab the main review text from each link.  
b.	Next run each review text through a tokenizer, and then try to NP-chunk it with a shallow parser.   
c.	You probably will have too many unknown words, owing to proper names of characters, actors, and so on that are not in your working dictionary. Make sure the main names that are relevant to the movies in your collection of reviews are added to the working lexicon, and then run the NP chunker again.  

3.	Output all the chunks in a single list for each review, and submit that output for this assignment. Also submit a brief written summary of what you did (describe your selection of genre, your source of reviews, how many you collected, and by what means).


In [2]:
import requests #needed to load the page for BS4
from bs4 import BeautifulSoup
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')
print(nlp.pipe_names)
nlp.pipeline

['tagger', 'parser', 'ner']


[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f6120f77f90>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f61210b2e50>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f61210b2ec0>)]

In [3]:
url1 = 'https://www.imdb.com/search/title/?genres=sci-fi&view=simple&explore=title_type,genres&title_type=movie&ref_=adv_explore_rhs'
url2 = 'https://www.imdb.com/search/title/?title_type=movie&genres=sci-fi&view=simple&start=51&explore=title_type,genres&ref_=adv_nxt'

In [4]:
html = requests.get(url1, stream=True)
page = BeautifulSoup(html.text, 'html.parser')
links = page.find_all('a href')

html2 = requests.get(url2, stream=True)
page2 = BeautifulSoup(html2.text, 'html.parser')
links2 = page.find_all('a href')

In [42]:
linklist = []
for a in page.find_all('a', href=True):
    #print ("Found the URL:", a['href'])
    if 'title/tt' in a['href']:
      #print(a)
      string = 'https://www.imdb.com' + a['href'] + 'reviews/?ref_=tt_ql_urv'
      linklist.append(string)

for a in page2.find_all('a', href=True):
    #print ("Found the URL:", a['href'])
    if 'title/tt' in a['href']:
      string = 'https://www.imdb.com' + a['href'] + 'reviews/?ref_=tt_ql_urv'
      linklist.append(string)      

In [36]:
reviews = pd.DataFrame(columns=['MovieLink', 'MovieNumber', 'ReviewNumber', 'Review'])
for i, val in enumerate(linklist):
  htmlr = requests.get(val, stream=True)
  pager = BeautifulSoup(htmlr.text, 'html.parser')
  #linksr = page.find_all('a href')

  #soup = BeautifulSoup(pager)
  mydivs = pager.findAll("div", {"class": "text show-more__control"})
  movie_containers = pager.find_all('div', class_ = 'parent')
  mytitle = movie_containers[0].text
  mytitle
  mytitle = mytitle.strip('\n')
  mytitle = mytitle.replace('\n','')
  mytitle = mytitle.replace('  ','')
  mytitle
  for i2, val2 in enumerate(mydivs):
    #print(i, i2, val, val2)
    reviewdict = {'Title': mytitle, 'MovieLink':val,'MovieNumber': i, 'ReviewNumber': i2,'Review':val2.text}
    df = pd.DataFrame(reviewdict, index=[0]) 
    reviews = reviews.append(df)

In [39]:
def getNounChunks(user_review):
    
    # create the doc object
    doc = nlp(user_review)
    
    # get a list of noun_chunks
    noun_chunks = list(doc.noun_chunks)
    
    # convert noun_chunks from span objects to strings, otherwise it won't pickle
    noun_chunks_strlist = [chunk.text for chunk in noun_chunks]
    
    return noun_chunks_strlist

In [40]:
reviews['noun_chunks'] = reviews['Review'].apply(getNounChunks)

In [43]:
pd.set_option('max_colwidth', 2500)

In [44]:
reviews.head(10)

,MovieLink,MovieNumber,ReviewNumber,Review,Title,noun_chunks
0,https://www.imdb.com/title/tt8332922/reviews/?ref_=tt_ql_urv,0,0,".......and if you enjoyed the first one then you'll certainly enjoy this one too.Most sequels that flop tend to do so because the powers that be think the audience simply want exponentially more of the same thing that was in the original. More bad guys, more explosions, much, much more of everything. Except story and depth and character development.A Quiet Place II, thankfully, doesn't fall into that trap. It starts almost exactly where it left off. Right at the point in fact, where we were left wanting to see more in the first film. Then it simply and effectively expands on the original.Overall I rate it as good as the first instalment, if not marginally better in places. If it has any fault at all, it's in the runtime which, IMHO, is a tad short. We definitely wanted to see more though and who knows, maybe there'll be a third one? Let's hope so.",A Quiet Place Part II(2020),"[you, the first one, you, this one, Most sequels, flop, the powers, the audience, the same thing, the original, More bad guys, more explosions, everything, story, depth, character development, A Quiet Place II, that trap, It, it, the point, fact, we, the first film, it, the original, I, it, the first instalment, places, it, any fault, it, the runtime, We, who, 's]"
0,https://www.imdb.com/title/tt8332922/reviews/?ref_=tt_ql_urv,0,1,"It's rare that we get movie movies that evolve upon their predecessor and leaves us craving more!Perfect amount of drama, suspenseful moments and action.John Kransinki has created such a fantastic ""monster world"" and I can't see anyone else directing these films. This is HIS baby..10/10, highly recommended.",A Quiet Place Part II(2020),"[It, we, movie movies, their predecessor, us, more!Perfect amount, drama, suspenseful moments, action, John Kransinki, such a fantastic ""monster world, I, anyone, these films, HIS baby]"
0,https://www.imdb.com/title/tt8332922/reviews/?ref_=tt_ql_urv,0,2,"Best cinema experience I've had in a while. A near full house with a near silent crowd (some loud eaters heh). Some may not of enjoyed the lack of plot (as in not much happens) and the ending, but I just see this film as a continuation of the story (as it's obvious there will be a third). You have to remember that it's right after the ending of the first one so the setting and time is the exact same, I think some people didn't get that. Millicent Simmonds shines in this film and Cillian Murphy alongside her only makes it better. Emily Blunt is still great but steps down this time and let's Millicent take the spotlight and she uses it perfectly. I didn't find the sequel as scary as the first film, but the scenes that had some jump scares in them were executed very well. Replacing the lack of scares in this film are big action pieces, which are great on the big screen, but I would of preferred more silent scenes where sound is used as a jump scare element rather than big explosions and loud sounds, but the action was still intense. Again, the sound design is perfect, especially when we're through Regans perspective and sound is cut out completely. Favourite scene was definitely the opening, big action scenes and a good origin to how the world is how it is. The only thing that bugged me was the theme of family has died down. It's now become 'every man for himself' almost, but we get a few moments that revolve around the theme of family, but you can see Regan and Marcus trying to be like their father during the film and trying to lead the family to safety. Highly recommend seeing this film, even if horror is not your taste you can appreciate the sound and production design.",A Quiet Place Part II(2020),"[Best cinema experience, I, a while, full house, a near silent crowd, some loud eaters, the lack, plot, the ending, I, this film, a continuation, the story, it, You, it, the ending, the first one, the setting, time, I, 

In [45]:
review_final = reviews.drop_duplicates(subset=['Title'])
review_final

,MovieLink,MovieNumber,ReviewNumber,Review,Title,noun_chunks
0,https://www.imdb.com/title/tt8332922/reviews/?ref_=tt_ql_urv,0,0,".......and if you enjoyed the first one then you'll certainly enjoy this one too.Most sequels that flop tend to do so because the powers that be think the audience simply want exponentially more of the same thing that was in the original. More bad guys, more explosions, much, much more of everything. Except story and depth and character development.A Quiet Place II, thankfully, doesn't fall into that trap. It starts almost exactly where it left off. Right at the point in fact, where we were left wanting to see more in the first film. Then it simply and effectively expands on the original.Overall I rate it as good as the first instalment, if not marginally better in places. If it has any fault at all, it's in the runtime which, IMHO, is a tad short. We definitely wanted to see more though and who knows, maybe there'll be a third one? Let's hope so.",A Quiet Place Part II(2020),"[you, the first one, you, this one, Most sequels, flop, the powers, the audience, the same thing, the original, More bad guys, more explosions, everything, story, depth, character development, A Quiet Place II, that trap, It, it, the point, fact, we, the first film, it, the original, I, it, the first instalment, places, it, any fault, it, the runtime, We, who, 's]"
0,https://www.imdb.com/title/tt6654210/reviews/?ref_=tt_ql_urv,2,0,"The main motive of the villian is, he is tired of reincarnated and by destroying all the lives in earth will stop it. But, he also got a bullet that will stuck the memory of the people in a chip so that he/she will not be reborn again. My question is why don't he put the bullet in his head and solve this problem in an easy way?",Infinite(2021),"[The main motive, the villian, he, all the lives, earth, it, he, a bullet, the memory, the people, a chip, she, My question, he, the bullet, his head, this problem, an easy way]"
0,https://www.imdb.com/title/tt0993840/reviews/?ref_=tt_ql_urv,4,0,"I miss these lists, they were fun!This movie was somewhat entertaining but also (and mostly) annoying:Spoilers, and in random order, enjoy:1: A leader zombie on top of a tall building, can look at a helicopter flying away, then give it 1 mile head start, go down to ground level, and still out run the helicopter on his dead horse.2: The more people talk about a nuke coming, and time running out, the more relaxed they will feel and the slower they will move.3: If a nuke is on the way and a helicopter comes to rescue you, you must still wait for the pilot to deliver a speech and give you permission to get on board, before moving at all.4: A piece of make shift metal for a helmet will act as a magnet for all bullets aimed at that person, and will never even have a mark on it, after 10+ hits.5: A woman who says she has a plan, while holding a zombie head, will keep staring at the helicopter taking off, probably because she remembered she didn't really have a plan.6: The Military generally needs their orders given 2-3 times before reacting, and they will then react very slowly and with seemingly no training.7: Having dark skin protects you from massive nuclear radiation in the same way it protects you better from the UV rays from the sun, compared to a very pale person.8: A young woman with no military training will have trouble shooting a zombie in the head at point blank range but can hit 8 running zombies in the head with 8 bullets, from a distance.9: A movie can last 2½ hours and still have very little character development.10: People will call a young man ""a kid"" and make fun of him, because he's the nerdy safe cracker, in spite him having huge biceps and broad shoulders, like we know all nerds generally have in real life.11: Noone will question why a billionaire wants a team to retrieve pocket cash for him.12: A random woman is important to save, except if she gets on a helicopter, then we don't even have to see if 